## Search in https://docs.python-mip.com/en/latest/classes.html

In [128]:
from mip import *

In [129]:
def ler_arquivo(nome_arquivo):
    try:
        with open(nome_arquivo, 'r') as arquivo:
            linhas = arquivo.readlines()
            tamanho_da_lista = len(linhas)
            lista_restricoes = []
            for i in range(tamanho_da_lista):
                linha = linhas[i].split()
                match i:
                    case 0:
                        quantidade_variaveis = int(linha[0])
                        quantidade_restricoes = int(linha[1])
                    case 1:
                        coeficientes_objetivo = []
                        for variaveis in range(quantidade_variaveis):
                            coeficientes_objetivo.append(int(linha[variaveis]))    
                    case _:
                        tamanho_da_linha = len(linha)
                        lista = []
                        for restricao in range(tamanho_da_linha):
                            lista.append(int(linha[restricao]))
                        lista_restricoes.append(lista)

        return (quantidade_variaveis, quantidade_restricoes, coeficientes_objetivo, lista_restricoes)
    
    except FileNotFoundError:
        print(f'O arquivo "{nome_arquivo}" não foi encontrado.')

In [165]:
def Criar_modelo(dados):
    quant_variaveis = dados[0]
    quant_restricoes = dados[1]
    coef_objetivo = dados[2]
    restricoes = dados[3]
    

    modelo = Model(sense=MAXIMIZE, solver_name=CBC)
    x = {i: modelo.add_var(var_type=CONTINUOUS, name=f'x_{i}', lb=0.0, ub=1.0) for i in  range(1,quant_variaveis+1)}
    modelo.objective = xsum(coef_objetivo[i-1]*x[i] for i in range(1,quant_variaveis+1))

    for i in range(quant_restricoes):
        tamanho_lista = len(restricoes[i])
        soma =  xsum(restricoes[i][j]*x[j+1] for j in range (tamanho_lista-1))
        modelo += soma <= restricoes[i][tamanho_lista-1]

    # modelo.write("modelo.lp") # salva modelo em arquivo
    # with open("modelo.lp") as f: # lê e exibe conteúdo do arquivo
    #     print(f.read())

    return modelo

In [185]:
class Node:
    def __init__(self, lb, ub, variable=None):
        self.lb = lb  # Limite inferior
        self.ub = ub  # Limite superior
        self.variable = variable  # Variável associada ao nó
        self.left_child = None  # Filho esquerdo (valor 0)
        self.right_child = None  # Filho direito (valor 1)

In [171]:
def update_best_solution(model, node, best_solution):
    # Resolve o subproblema associado ao nó
    model.optimize()
    # Verifica se há uma solução e se é melhor que a atual melhor solução
    if model.num_solutions > 0 and model.objective_value > best_solution[0]:
        # Atualiza a melhor solução encontrada
        best_solution[0] = model.objective_value

In [173]:
def explore_tree(model, node_list, best_solution):
    new_node_list = []  # Lista para armazenar os novos nós gerados
    
    for node in node_list:
        if node.lb >= node.ub:
            continue  # Não é necessário dividir um nó que já atingiu um limite
        # Calcula o ponto médio entre os limites inferiores e superiores
        mid_point = (node.lb + node.ub) / 2
        # Cria dois novos nós com base no ponto médio
        left_child = Node(node.lb, mid_point)
        right_child = Node(mid_point, node.ub)
        # Adiciona os novos nós à lista
        new_node_list.extend([left_child, right_child])
        # Atualiza a melhor solução encontrada, se necessário
        update_best_solution(model, left_child, best_solution)
        update_best_solution(model, right_child, best_solution)
    
    # Retorna a lista de novos nós gerados
    return new_node_list

In [174]:
def solve_subproblem(model, node):
    # Define os limites inferiores e superiores das variáveis de decisão de acordo com o nó
    for var, value in zip(model.vars, node.lb, node.ub):
        var.lb = value
        var.ub = value
    # Resolve o modelo associado ao nó
    model.optimize()
    # Retorna a solução ótima, se houver
    if model.num_solutions > 0:
        return model.objective_value
    else:
        return None

In [176]:
def prune_tree(node_list, best_solution):
    pruned_node_list = []  # Lista para armazenar os nós podados
    
    for node in node_list:
        # Se o limite inferior for maior ou igual à melhor solução encontrada até agora, mantenha o nó
        if node.lb >= best_solution[0]:
            pruned_node_list.append(node)
    
    # Retorna a lista de nós podados
    return pruned_node_list

In [182]:
def who_is_closer(modelo):
    var_prox_05 = None
    melhor_dist = 1000

    for v in modelo.vars:
        dist = abs(v.x - 0.5)

        if dist < melhor_dist:
            var_prox_05 = v
            melhor_dist = dist
 
    return var_prox_05

In [177]:
def should_stop(node_list):
    return len(node_list) == 0

In [183]:
def branch_and_bound(model):
    root_node = Node(0, 1)  # Raiz da árvore de busca
    best_solution = -float('inf')  # Melhor solução encontrada
    
    node_list = [root_node]  # Inicializa a lista de nós com a raiz
    
    while node_list:
        # Escolhe o nó mais próximo de 0.5
        current_node = min(node_list, key=lambda node: abs(node.lb + node.ub - 1) / 2)
        node_list.remove(current_node)  # Remove o nó escolhido da lista
        
        # Define os limites inferiores e superiores das variáveis associadas ao nó
        if current_node.variable:
            current_node.variable.lb = current_node.lb
            current_node.variable.ub = current_node.ub
        
        # Resolve o modelo associado ao nó
        model.optimize()
        
        # Atualiza a melhor solução encontrada
        if model.num_solutions > 0 and model.objective_value > best_solution:
            best_solution = model.objective_value

        H = who_is_closer(model)
        
        # Cria os nós filhos com valores 0 e 1
        current_node.left_child = Node(current_node.lb, current_node.lb + (current_node.ub - current_node.lb) / 2, H)
        current_node.right_child = Node(current_node.left_child.ub, current_node.ub, H)
        
        # Adiciona os nós filhos à lista de nós
        node_list.extend([current_node.left_child, current_node.right_child])
    
    return best_solution


In [184]:
# Ler todos os aquivos do acervo ()
for i in range(1, 2):
    dados_problema = ler_arquivo(f"teste{i}.txt")
    
    modelo = Criar_modelo(dados_problema)

    print(branch_and_bound(modelo))

AttributeError: 'Node' object has no attribute 'variable'

In [178]:
def branch_and_bound(model):
    # Inicializa a lista de nós com a raiz da árvore de busca
    root_node = Node(0, 1)
    node_list = [root_node]
    # Inicializa a melhor solução encontrada
    best_solution = [-float('inf')]

    # Loop principal do algoritmo Branch and Bound
    while not should_stop(node_list):
        # Explora a árvore de busca e gera novos nós
        node_list = explore_tree(model, node_list, best_solution)
        # Poda os nós que não levam a soluções promissoras
        node_list = prune_tree(node_list, best_solution)

    # Retorna a melhor solução encontrada
    return best_solution[0]

In [ ]:
# FUNÇÕES A SEREM EXPLORADAS

# cuts_generator:
    # Um gerador de cortes é um ConstrsGenerator objeto que recebe uma solução fracionária e tenta gerar uma ou mais restrições (cortes) para removê-la. 
    # O gerador de cortes é chamado em cada nó da árvore branch-and-cut onde é encontrada uma solução que viola a restrição de integralidade de uma ou mais variáveis.

# generate_cuts( cut_types = Nenhum , profundidade = 0 , npass = 0 , max_cuts = 8192 , min_viol = 0,0001 )

# max_nodes número máximo de nós a serem explorados na árvore de busca

# optimize( max_seconds = inf , max_nodes = 1073741824 , max_solutions = 1073741824 , max_seconds_same_incumbent = inf , max_nodes_same_incumbent = 1073741824 , relax = False )

# remove(objetos)

# var_by_name( nome ) - Pesquisa uma variável pelo seu nome


In [132]:
def solve(modelo):
    status = modelo.optimize()

    print("Status = ", status)
    print(f"Solution value  = {modelo.objective_value:.2f}\n")
    
    print("Solution:")
    for v in modelo.vars:
        print(f"{v.name} = {v.x:.2f}")

In [138]:
def branch_bound(modelo):
    solve(modelo)

    if modelo.objective_value != round(modelo.objective_value):
        modelo.add_constr(who_is_closer(modelo).name == 0)

        print("--------------")
        print(modelo.constrs[-1])
        solve(modelo)

    if modelo.objective_value != round(modelo.objective_value):
        modelo.remove(modelo.constrs[-1])
        modelo.add_constr(who_is_closer(modelo).name == 1)

        print("--------------")
        print(modelo.constrs[-1])
        solve(modelo)

In [ ]:
def adicionar_restriçao_menor(variavel,restricao,nome):
    model.add_constr(variavel <= restricao,nome)

def adicionar_restriçao_maior(variavel,restricao,nome):
    model.add_constr(variavel >= restricao,nome)

In [146]:
def is_all_integer(model):
    for var in model.vars:
        if not var.x.is_integer():
            return False
    return True


In [ ]:
for i in range(1, 2):
    dados_problema = ler_arquivo(f"teste{i}.txt")
    
    model = Criar_modelo(dados_problema)

    model.optimize()

    # Modelo com o melhor valor de Z
    melhor_no = Model()

    if is_all_integer(model) and model.objective_value > melhor_no.objective_value:
        melhor_no = model
    if not all_integer(model):
        var_no = who_is_closer(model)

        model.add_constr(var_no == 0)




In [151]:
melhor_no = Model()
print(1 > melhor_no.objective_value)

TypeError: '>' not supported between instances of 'int' and 'NoneType'

In [ ]:
# Ler todos os aquivos do acervo ()
for i in range(1, 2):
    dados_problema = ler_arquivo(f"teste{i}.txt")
    
    modelo = Criar_modelo(dados_problema)

    solve(modelo)

    who_is_closer(modelo)


    # X3 = 0. 
    MELHOR_NO = modelo

    # X3 = 1 e X5 = 1.
    MELHOR_NO = modelo

    # SOLUCIONA E SELECIONA X9 
    # X9 = 0 
        # SOLUNCIONA E SELECIONA X3 
        # X3 = 0 === SOLUÇÃO (SALVAR EM MELHOR_NO) 

        # REMOVE RESTRIC X3= 0
        # X3 = 1
            # SOLUCIONA E SELECIONA X5
            # X5 = 0 == N INTEIRO E MELHOR_NO.objective_value > modelo.objective_value

            # REMOVE RESTRIC X5 = 0
            # X5 = 1 === SOLUÇÃO E (MELHOR_NO.objective_value > modelo.objective_value) = FALSO
                # SALVAR EM MELHOR_NO 

            # modelo.remove(modelo.constrs[-1])
        
        # modelo.remove(modelo.constrs[-1])

    # modelo.remove(modelo.constrs[-1])

    # X9 = 1
        # 


    # node = modelo.add_node()

    # cuts = my_cut_generator(modelo)

    # for cut in cuts:
    #     modelo += cut

    # solve(modelo)
    # print()
    # print(modelo.num_solutions)
    # print(modelo.constrs[-1])

    # x_value = modelo.x.x

    # modelo.cuts_generator = my_cut_generator

    # for cut in cuts:
    #     modelo += cut


    # modelo.var_by_name(who_is_closer(modelo))
    # a =modelo.var_by_name(who_is_closer(modelo).name)


    # print(modelo.vars[1])



    # a = my_cut_generator(modelo)



    # m.my_cut_generator(modelo)
    # m.write("model.lp") # salva modelo em arquivo
    # with open("model.lp") as f: # lê e exibe conteúdo do arquivo
    #     print(f.read())

    # model.generate_cuts(my_cut_generator)
    # modelo.cuts_generator = my_cut_generator

    # modelo.add_constr(my_cut_generator)

    # solve(modelo)




In [144]:
def solve(modelo):
    status = modelo.optimize()

    print("Status = ", status)
    print(f"Solution value  = {modelo.objective_value:.2f}\n")
    
    print("Solution:")
    for v in modelo.vars:
        print(f"{v.name} = {v.x:.2f}")

In [169]:
for i in range(2, 3):
    dados_problema = ler_arquivo(f"teste{i}.txt")
    
    # Criar modelo possui  x = {i: modelo.add_var(var_type=BINARY, name=f'x_{i}', lb=0.0) for i in  range(1,quant_variaveis+1)}
    modelo = Criar_modelo(dados_problema)

    modelo.optimize()

    modelo.add_constr(who_is_closer(modelo) == 1)
    modelo.add_constr(modelo.vars[1] == 1)
    # modelo.add_constr(modelo.vars[2] == 1)

    def foda(modelo):
        for var in modelo.vars:
                if not var.x == round(var.x):
                    return False
        return True


    print(modelo.objective_value)
    a = foda(modelo)
    print("HHHHHHHHHHHHHHHHHHHH ", a)
    # print(is_all_integer(modelo))


    # print(who_is_closer(modelo))

    # modelo.write("modelo.lp") # salva modelo em arquivo
    # with open("modelo.lp") as f: # lê e exibe conteúdo do arquivo
    #     print(f.read())

    
    solve(modelo)

    print(modelo.objective_value:.2f)


    

SyntaxError: invalid decimal literal (3782745176.py, line 35)

In [ ]:
from mip import Model, maximize, BINARY

class Node:
    def __init__(self, lb, ub):
        self.lb = lb  # Limite inferior
        self.ub = ub  # Limite superior
        self.value = None  # Valor da variável associada ao nó
        self.left_child = None  # Filho esquerdo
        self.right_child = None  # Filho direito

def branch_and_bound(model, root, best_solution):
    if root.lb >= root.ub:
        return
    x_val = (root.lb + root.ub) / 2
    # Calcula a diferença em relação a 0.5 para determinar o próximo nó
    diff_to_half = abs(0.5 - x_val)
    # Cria o filho esquerdo com x = 0
    left_child = Node(root.lb, x_val)
    model += x <= x_val
    model.optimize()
    if model.num_solutions > 0 and model.objective_value > best_solution[0]:
        best_solution[0] = model.objective_value
    branch_and_bound(model, left_child, best_solution)
    model.remove(model.vars[-1])
    # Cria o filho direito com x = 1
    right_child = Node(x_val + 1, root.ub)
    model += x >= x_val + 1
    model.optimize()
    if model.num_solutions > 0 and model.objective_value > best_solution[0]:
        best_solution[0] = model.objective_value
    branch_and_bound(model, right_child, best_solution)
    model.remove(model.vars[-1])
    # Atualiza os filhos do nó atual
    root.left_child = left_child
    root.right_child = right_child

# Variável de decisão
for i in range(2, 3):
    dados_problema = ler_arquivo(f"teste{i}.txt")
    model = Criar_modelo(dados_problema)


x = model.add_var(name='x', var_type=BINARY)

# Melhor solução encontrada
best_solution = [-float('inf')]

# Raiz da árvore de busca
root = Node(0, 1)

# Criação dos nós na árvore de busca
branch_and_bound(model, root, best_solution)


# Variável de decisão
x = model.add_var(name='x', var_type=BINARY)

# Criação dos nós na árvore de busca
branch_and_bound(model, root, best_solution)

# Imprime a melhor solução encontrada
print("Melhor solução encontrada:", best_solution[0])
